## `lab12`—The Traveling Salesman

❖ Objectives

-   Explore optimization using a classic problem.

In [ ]:
# Import basic libraries.
import math
import itertools
import requests

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams[ 'figure.figsize' ] = (15,2)

##  Visiting Cities

A classic optimization problem, the Traveling Salesman Problem (TSP) posits that a salesman would like to visit every city in a set.  The salesman would prefer to spend as little time on the road as possible, so he wants to optimize his route to be the shortest possible.

For instance, for three cities, it is clear that one of these paths takes significantly less time than the others:

![](./img/tsp-3.png)

For more than a handful of cities, however, the number of possible paths explodes and the problem becomes computationally challenging. For the number of cities $n$, the amount of possible paths the salesman can take through the cities is equal to $n! = \prod_{i=1}^{n} i$, making brute-force solutions for a lot of cities unmanagable for even the fastest computers.

![](./img/tsp-many.png)

Brute-force optimization thus works well for a smaller number of cities, but for a bigger set of cities we need heuristic optimization strategies. Run the cell below to show how many possible paths the computer needs to examine through brute force for different values of $n$ to find the optimal route:

In [ ]:
print(*("n = {:3g}: {:.4g} possible paths".format(i, math.factorial(i)) \
        for i in itertools.chain(range(2, 21, 2), range(30, 60, 10), range(75, 101, 25))), sep=",\n")

The cities we wish to visit are available online as a JSON resource including city names and locations (thus distances). *JSON* is a standard Internet data format which Python can easily convert into a dictionary.  The `requests` library fortunately supports accessing this type, so we can use the `.json()` method (*not* attribute like `.text`) to retrieve the data from a URL.

-   Compose a function `get_cities` which returns the dictionary from the `json()` method at the URL `go.cs.illinois.edu/cs101-tspdat1011` (just copy and paste it).

In [ ]:
# Compose your function `get_cities` here.

In [ ]:
# You may test your function here if desired.

In [ ]:
# your code should pass these tests---do NOT edit this cell
test_city_locations = get_cities()

print( *sorted( test_city_locations.keys() ),sep=', ' )

assert len(test_city_locations) == 17, "Number of cities is incorrect."
assert 'El Dorado' in test_city_locations, "Names of cities are incorrect."
assert test_city_locations['El Dorado'] == [79, 97], "Locations of cities are incorrect."
print('Success!')

In order to tell how long the path is it's possible to evaluate if we've found the best one, we need to come up with functions that can return the distance between two cities, as well as a function that will compute the distance of an entire path.

-   Compose a function `city_distance` which, given a dictionary and two city names, computes the distance between them according to the standard euclidean distance metric, $\sqrt{(x_1-x_0)^2 + (y_1-y_0)^2}$.  The dictionary is expected to be the output of `get_cities`, of course.

In [ ]:
# Compose your function `city_distance` here.

In [ ]:
# You may test your function here if desired.

In [ ]:
# your code should pass these tests---do NOT edit this cell
from numpy import isclose
test_city_locations = get_cities()

assert isclose(city_distance( test_city_locations,'Avalon','Gotham' ), 16.1245), "Distance calculation is incorrect."
assert isclose(city_distance( test_city_locations,'Gotham','Quahog' ), 74.2765), "Distance calculation is incorrect."
print('Success!')

-   We require a function `path_distance` which accepts a dictionary and a list of city names.  `path_distance` should return the length of the total path.  The following candidate function has been provided by a colleague of yours known for shoddy coding.  You suspect that it may have bugs.  Find and correct any of them in the code below.

In [ ]:
def path_distance( city_dict,city_path ):
    
    # Copy the incoming path since we'll modify it.
    path1 = city_path[ : ]
    path2 = city_path[ : ]
    
    del path2[ 0 ]
    del path1[ len(path2)-1 ]
    
    # Now we have two lists, path1 being the starting city of each step
    # and path2 being the ending city of each step:
    
    # Loop over the city pairs and sum up the total distance traveled:
    total_d = 0.0
    for city1,city2 in zip( path1,path1 ):
        d = city_distance( city_dict,city1,city2 )
        total_d += d
    
    return d

In [ ]:
# your code should pass these tests---do NOT edit this cell
from numpy import isclose
test_city_locations = get_cities()

assert path_distance( test_city_locations,['Avalon', 'Gotham', 'Quahog'] ) == \
       city_distance( test_city_locations,'Avalon', 'Gotham' ) + \
       city_distance( test_city_locations,'Gotham', 'Quahog' ), \
    "Path calculation is incorrect."
print('Success!')

##  Finding a Way

Now that we have ways of evaluating candidate routes, let's take a look at algorithms.

First, we have our old friend the brute-force search.

-   Given a set of cities and locations (we have that from `get_cities`), find the shortest route between them using the brute-force algorithm.  You may use the `itertools.permutations` function to comprehensively review the cities, but any candidate path needs to include all of the cities once. Be aware that `itertools.permutations` gives tuples, not lists!
    
    Compose a function `brute_force_tsp` which implements the brute-force algorithm as just described.  `brute_force_tsp` accepts the standard dictionary containing cities as well as a list of cities which need to be visited and returns the globally optimal path (shortest distance traveled).

In [ ]:
# Compose your function `brute_force_tsp` here.

In [ ]:
# You may test your function here if desired.

In [ ]:
# your code should pass these tests---do NOT edit this cell
from numpy import isclose
test_city_locations = get_cities()

given_cities = [ 'Avalon','Bedrock','El Dorado','Emerald City','Gotham','Hill Valley' ]
path = brute_force_tsp( test_city_locations,given_cities )
assert type(path) == type(list()), "Path returned is not a list, but {}".format(type(path))
assert np.isclose( path_distance(test_city_locations,path ), 219.5677 ), "Path calculated is not the shortest."
print('Success!')

Given a set much larger than the six cities in the test case, brute-force searching will take too long.  We will employ another algorithm, a heuristic optimization technique called *branch and bound*. Branch and bound works by examining the best solution found so far, and if it is possible to determine that the candidate solution cannot possibly beat the *bound*, or best solution we have already, we drop all the computation down that *branch*.

To begin, we need to approximate a good solution to the TSP which we can set as our *initial bound*. One way to do that is using the nearest-neighbor heuristic:  start with the first city, and then build the path by always traveling to the closest unvisited city.

Let's take a look at the difference between a nearest-neighbor guess and the optimal solution for a certain case.

![](./img/tsp-greedy.png)

You can see that the nearest-neighbor heuristic isn't optimal (it's analogous to a hill-climbing solution), but it doesn't do *too* poorly.  Examine the implementation below (it does a couple of new things that we haven't examined yet), and evaluate it to make sure that `nearest_neighbor_heuristic` can run.

In [ ]:
def nearest_neighbor_heuristic(cities, must_visit):
    nn_heuristic = must_visit[ :1 ]  # The heuristic path our function builds
    nn_unvisited = must_visit[ 1: ]  # The cities yet to be added to our path
    
    # While there are cities that haven't been visited...
    while nn_unvisited:
        curr_city = nn_heuristic[-1] # Get the city that the salesman is in
        
        # Find the closest unvisited city
        next_dist, next_i = min((city_distance(cities, curr_city, next_city), city_i)
                 for city_i, next_city in enumerate(nn_unvisited))
        
        # Append that city to our path, and remove it from our unvisited
        nn_heuristic.append(nn_unvisited.pop(next_i))
        
    return (path_distance(cities, nn_heuristic), nn_heuristic)

With our initial bound added, we can begin coding the branch-and-bound. Your zealous colleague has once again composed a useful function `branch_bound_tsp` to assist in this process, but you suspect that there may be some issues with the function.  Examine `branch_bound_tsp` for bugs, correct any that you notice, and solve the TSP.

In [ ]:
def branch_bound_tsp( cities,must_visit ):
    # cities is 
    
    # Set our best solution as the nearest-neighbor approximation
    best_sol = nearest_neighbor_heuristic( cities,must_visit ) 
    
    # Create a list of incomplete solutions - tuples of distances and paths, one starting in each city
    sol_list = [ ]
    for city in must_visit:
        sol_list.append( ( 0,[ city ] ) )
    
    while sol_list:
        # Get our best distance and path from our best solution
        best_dist, best_path = best_sol
        
        # Get our current solution from our list of solutions
        # (list.pop removes the last item from a list and returns it)
        # (this line is okay)
        curr_dist, curr_path = sol_list.pop() 
        
        # Build a list of unvisited cities for our current path
        curr_remaining = [ ]
        for city in must_visit:
            if city not in curr_path:
                curr_remaining.append( city )
        
        # If there are no remaining cities, this path is complete.
        # Compare it to our current best solution and see which is better:
        if len( curr_remaining ) = 0:
            best_sol = min( best_sol,( curr_dist,curr_path ) )
        
        # If our current distance is worse than the best solution, adding cities won't help.
        # This solution doesn't work, so just skip to the next iteration of the loop.
        if curr_dist >= best_dist:
            continue
        
        # Otherwise, we may want to examine this path further. Add the next steps to the
        # start of the list so we can examine them in later iterations of the for loop.
        for i, city in enumerate( curr_remaining ):
            next_dist = curr_dist + city_distance( cities,curr_path[ -1 ],city )
            next_path = curr_path.append( city )
            
            # If the distance is lower than the best distance, append the solution to
            # the start of the list.
            if next_dist < best_dist:
                sol_list.insert( 1,( next_dist,next_path ) )
    
    # Return the best path.
    return best_sol[ 1 ]

In [ ]:
# You may test your function here if desired.

In [ ]:
# your code should pass these tests---do NOT edit this cell
from numpy import isclose
from time import time
test_city_locations = get_cities()
given_cities = list(test_city_locations.keys())[:10]

print("Running branch and bound (n = 10)...")
start_bb = time()
branch_bound_path = branch_bound_tsp(test_city_locations, given_cities)
end_bb = time()
print("Branch and bound time: ", end_bb - start_bb, "\n")

print("Running brute force (n = 10)...")
start_brute = time()
brute_force_path = brute_force_tsp(test_city_locations, given_cities)
end_brute = time()
print("Brute force time: ", end_brute - start_brute)

assert end_brute-start_brute - 15 > end_bb-start_bb, "Branch and bound is not substantially faster than brute force."
assert path_distance( test_city_locations,brute_force_path ) == path_distance( test_city_locations,branch_bound_path ), "Branch and bound path does not return the same distance as brute-force path."

print('Success!')

This is a group lab, so you should write down the names and NetIDs of your group in the cell below.  Then **save** before submitting.

In [ ]:
'''
write them here
'''